# Issue a Credential using BIP 322 Signatures

This notebook outlines how a Bitcoin address can be used to sign a verifiable credential using BIP 322 signatures explored in other notebooks. Initially, this will just look at using a bitcoin address as the issuer although eventually this should transition to a did:btcr (v2?)

## Notebook Author

* Will Abramson
* [Legendary Requirements](http://legreq.com/)
* Contact: will@legreq.com

## Acknowledgements

This work was funded by Ryan Grant and Digital Contract Design. Thanks also go to Joe Andrieu, Kalle Alm, Pieter Wuille and Jimmy Song for engaging with and supporting various aspects of this work.

In [2]:
cd ..

/home/will/work/LegendaryRequirements/clients/dcd/bip0322-sigs


## Steps

1. Define VC
2. Canonicalise VC (Do we need this?)
3. BIP 322 sign VC payload
4. Attach signature to VC as proof

## 1. Define VC


Issuer needs to decide:

* Contents of credential
* Schema of credential
* Key pair and associated identifier they will be signing with
* Identifier to use to the credential subject


### 1.1. TODO: How do we identify the issuer

Should be use a did:btcr? Would need to be able to resolve this to a DID document and incombination with the proof data fetch the bitcoin address required to verify the signature.

In [3]:
unsigned_vc = {
  
  "@context": [
    "https://www.w3.org/2018/credentials/v1",
    "https://www.w3.org/2018/credentials/examples/v1"
  ],
  
  "id": "http://example.edu/credentials/1872",
  


  "type": ["VerifiableCredential", "AlumniCredential"],
  

  # TODO: How identifier issuer
  # Should be a did:btcr?
  "issuer": "https://example.edu/issuers/565049",
  


  "issuanceDate": "2010-01-01T19:23:24Z",
  


  "credentialSubject": {
    

    # TODO: How identify credential subject
    "id": "did:example:ebfeb1f712ebc6f1c276e12ec21",
    

    "alumniOf": {
      "id": "did:example:c276e12ec21ebfeb1f712ebc6f1",
      "name": [{
        "value": "Example University",
        "lang": "en"
      }, {
        "value": "Exemple d'Université",
        "lang": "fr"
      }]
    }
  },
  


}

## 2. Canonicalize VC

Not exactly sure what this contains. Need a way to determinstically turn JSONLD VC object into a message to be signed.

I think only need to canonicalize if rendering data in JSONLD VC from other sources? RDF Dataset?

- https://json-ld.github.io/rdf-dataset-canonicalization/spec/
- https://www.w3.org/TR/vc-data-model/#json-ld

In [4]:
# Naive approach?

import json
from buidl.helper import str_to_bytes

vc_payload = json.dumps(unsigned_vc)
print(str_to_bytes(vc_payload))

b'{"@context": ["https://www.w3.org/2018/credentials/v1", "https://www.w3.org/2018/credentials/examples/v1"], "id": "http://example.edu/credentials/1872", "type": ["VerifiableCredential", "AlumniCredential"], "issuer": "https://example.edu/issuers/565049", "issuanceDate": "2010-01-01T19:23:24Z", "credentialSubject": {"id": "did:example:ebfeb1f712ebc6f1c276e12ec21", "alumniOf": {"id": "did:example:c276e12ec21ebfeb1f712ebc6f1", "name": [{"value": "Example University", "lang": "en"}, {"value": "Exemple d\'Universit\\u00e9", "lang": "fr"}]}}}'


## 3. Sign VC

**Code for sign_message can be found in [message.py](../../edit/src/message.py)**

In [5]:
from buidl.ecc import PrivateKey
private_key_wif = "L4qmTC36US6tngrqbrW6vffZbxUVSfipQqXxdJw8S5HqyxNebaMP"
private_key = PrivateKey.parse(private_key_wif)

In [15]:
### TODO: Note using pay-to-witness-public-key-hash address. Should be able to in theory use ANY btc script

In [14]:
address = private_key.point.p2wpkh_address()
address

'bc1qz52z3pe9fg3qxv9n6yhxgj7rcn8wsvpq56v9ck'

In [11]:
from src.message import sign_message, MessageSignatureFormat

vc_sig = sign_message(MessageSignatureFormat.SIMPLE, private_key, address, vc_payload)
print("\n\nBIP 322 Simple Signature on VC", vc_sig)

Is taproot False

tx: cde29e32abd0054a2e4bf3dde55819795c338944816e27ab107521a2f8fc94e4
version: 0
locktime: 0
tx_ins:
bd3fbf362c69f5fd221fa26ee8bdb9b6b019722595c8ebc66ca70a0e6350d931:0
tx_outs:
0:OP_RETURN 

txIn tap script None
txIn witness 3045022100a819e2856ee7597007143ebb4cb3d711c183ad28bee7934ef53c5a5e5192b86802201f639592fa0864166419ce12dc18deb04808e541e2db05880f8b555ede1be7b301 03c8822574d7c7055ade7c7dd598dd5af47e72354b56e3fa1e2c2c3cf025f6cf31 
True
Combined Script OP_0 15142887254a220330b3d12e644bc3c4cee83020 


BIP 322 Simple Signature on VC AkgwRQIhAKgZ4oVu51lwBxQ+u0yz1xHBg60ovueTTvU8Wl5RkrhoAiAfY5WS+ghkFmQZzhLcGN6wSAjlQeLbBYgPi1Ve3hvnswEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8x


## 4. Attach Signature to VC as proof

In the BIP322 Signature Suite (TBD) we would define three proof types?

* BIP322LegacySignature2022 
* BIP322SimpleSignature2022
* BIP322FullSignature2022

**Note: Actually perhaps it only requires a BIP322Signature2022 definition. From the signature provided the implementation determines whether it is LEGACY,SIMPLE or FULL**

These types would define how the proof JSON should be structured and interpretted

### 4.1. TODO: what are the acceptable verification methods? How would a verifier be able to resolve to a bitcoin address usable to verify a BIP 322 signature

In [12]:


proof = {

    "type": "BIP322Signature2022",
    


    "created": "2022-05-23T21:19:10Z",
    


    "proofPurpose": "assertionMethod",
    

    
    # This verification method would identify a UTXO on the bitcoin network?
    "verificationMethod": "https://example.edu/issuers/565049#key-1",
    


    "bip322_sig": vc_sig
}

In [13]:
signed_vc = unsigned_vc
signed_vc["proof"] = proof
print(json.dumps(signed_vc))

{"@context": ["https://www.w3.org/2018/credentials/v1", "https://www.w3.org/2018/credentials/examples/v1"], "id": "http://example.edu/credentials/1872", "type": ["VerifiableCredential", "AlumniCredential"], "issuer": "https://example.edu/issuers/565049", "issuanceDate": "2010-01-01T19:23:24Z", "credentialSubject": {"id": "did:example:ebfeb1f712ebc6f1c276e12ec21", "alumniOf": {"id": "did:example:c276e12ec21ebfeb1f712ebc6f1", "name": [{"value": "Example University", "lang": "en"}, {"value": "Exemple d'Universit\u00e9", "lang": "fr"}]}}, "proof": {"type": "BIP322SimpleSignature2022", "created": "2022-05-23T21:19:10Z", "proofPurpose": "assertionMethod", "verificationMethod": "https://example.edu/issuers/565049#key-1", "bip322_sig": "AkgwRQIhAKgZ4oVu51lwBxQ+u0yz1xHBg60ovueTTvU8Wl5RkrhoAiAfY5WS+ghkFmQZzhLcGN6wSAjlQeLbBYgPi1Ve3hvnswEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8x"}}


### BIP322 Legacy Example : Sign using p2pkh 

In [20]:
p2pkh_address = private_key.point.address()
p2pkh_address

'12vTLer5v6RyMpnxetu7mGxtveRsVSps5R'

In [27]:
vc_sig = sign_message(MessageSignatureFormat.LEGACY, private_key, p2pkh_address, vc_payload)
print("\n\nBIP 322 LEGACY Signature on VC : \n", vc_sig)



BIP 322 LEGACY Signature on VC : 
 MEUCIQDLQvF+0Dg0uruDJCydBwCV8vC03Y4VagBnEzKflu1HNQIgSKBu1Q6xWlQSZZVxUPgzVnTrXakcdBvV4BdWS4Oi/Qk=


In [28]:
proof = {

    "type": "BIP322Signature2022",
    


    "created": "2022-05-23T21:19:10Z",
    


    "proofPurpose": "assertionMethod",
    

    
    # This verification method would identify a UTXO on the bitcoin network?
    "verificationMethod": "https://example.edu/issuers/565049#key-1",
    


    "bip322_sig": vc_sig
}

In [30]:
signed_vc = unsigned_vc
signed_vc["proof"] = proof
print(json.dumps(signed_vc))

{"@context": ["https://www.w3.org/2018/credentials/v1", "https://www.w3.org/2018/credentials/examples/v1"], "id": "http://example.edu/credentials/1872", "type": ["VerifiableCredential", "AlumniCredential"], "issuer": "https://example.edu/issuers/565049", "issuanceDate": "2010-01-01T19:23:24Z", "credentialSubject": {"id": "did:example:ebfeb1f712ebc6f1c276e12ec21", "alumniOf": {"id": "did:example:c276e12ec21ebfeb1f712ebc6f1", "name": [{"value": "Example University", "lang": "en"}, {"value": "Exemple d'Universit\u00e9", "lang": "fr"}]}}, "proof": {"type": "BIP322Signature2022", "created": "2022-05-23T21:19:10Z", "proofPurpose": "assertionMethod", "verificationMethod": "https://example.edu/issuers/565049#key-1", "bip322_sig": "MEUCIQDLQvF+0Dg0uruDJCydBwCV8vC03Y4VagBnEzKflu1HNQIgSKBu1Q6xWlQSZZVxUPgzVnTrXakcdBvV4BdWS4Oi/Qk="}}


## BIP 322 FULL Example

In [34]:
# Note: if trust SIMPLE with p2pkh_address it is converted to FULL, because no witness to encode
full_vc_sig = sign_message(MessageSignatureFormat.SIMPLE, private_key, p2pkh_address, vc_payload)
print("\n\nBIP 322 FULL Signature on VC : \n", full_vc_sig)

Is taproot False

tx: 5ae3323662b37c10f0536fb5dce82f59dc90c14eebf71fd1524cfae30d7bd080
version: 0
locktime: 0
tx_ins:
ff877b9414f93a391d14fb06102e2a34accdeee14affb056ec8c2bd51331ffaa:0
tx_outs:
0:OP_RETURN 

txIn tap script None
txIn witness 
True
Combined Script 30440220643027ce64e62573cee521ee4c8b7a35b82545be64b4e5daf72e0384db222d3b022074b35ddedd5b92fe9ac199b9eeeada20040cb8a4fc5b0af53588ca7a3b7da77b01 03c8822574d7c7055ade7c7dd598dd5af47e72354b56e3fa1e2c2c3cf025f6cf31 OP_DUP OP_HASH160 15142887254a220330b3d12e644bc3c4cee83020 OP_EQUALVERIFY OP_CHECKSIG 


BIP 322 FULL Signature on VC : 
 AAAAAAABAar/MRPVK4zsVrD/SuHuzaw0Ki4QBvsUHTk6+RSUe4f/AAAAAGpHMEQCIGQwJ85k5iVzzuUh7kyLejW4JUW+ZLTl2vcuA4TbIi07AiB0s13e3VuS/prBmbnu6togBAy4pPxbCvU1iMp6O32newEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8xAAAAAAEAAAAAAAAAAAFqAAAAAAA=


In [35]:
proof = {

    "type": "BIP322Signature2022",
    


    "created": "2022-05-23T21:19:10Z",
    


    "proofPurpose": "assertionMethod",
    

    
    # This verification method would identify a UTXO on the bitcoin network?
    "verificationMethod": "https://example.edu/issuers/565049#key-1",
    


    "bip322_sig": full_vc_sig
}

In [36]:
signed_vc = unsigned_vc
signed_vc["proof"] = proof
print(json.dumps(signed_vc))

{"@context": ["https://www.w3.org/2018/credentials/v1", "https://www.w3.org/2018/credentials/examples/v1"], "id": "http://example.edu/credentials/1872", "type": ["VerifiableCredential", "AlumniCredential"], "issuer": "https://example.edu/issuers/565049", "issuanceDate": "2010-01-01T19:23:24Z", "credentialSubject": {"id": "did:example:ebfeb1f712ebc6f1c276e12ec21", "alumniOf": {"id": "did:example:c276e12ec21ebfeb1f712ebc6f1", "name": [{"value": "Example University", "lang": "en"}, {"value": "Exemple d'Universit\u00e9", "lang": "fr"}]}}, "proof": {"type": "BIP322Signature2022", "created": "2022-05-23T21:19:10Z", "proofPurpose": "assertionMethod", "verificationMethod": "https://example.edu/issuers/565049#key-1", "bip322_sig": "AAAAAAABAar/MRPVK4zsVrD/SuHuzaw0Ki4QBvsUHTk6+RSUe4f/AAAAAGpHMEQCIGQwJ85k5iVzzuUh7kyLejW4JUW+ZLTl2vcuA4TbIi07AiB0s13e3VuS/prBmbnu6togBAy4pPxbCvU1iMp6O32newEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8xAAAAAAEAAAAAAAAAAAFqAAAAAAA="}}
